In [5]:
# Importar las librerías necesarias
import pandas as pd

# Cargar los archivos CSV en DataFrames
eventos = pd.read_csv('/workspace/eventos.csv')
aficionados = pd.read_csv('/workspace/aficionados.csv')
promociones = pd.read_csv('/workspace/promociones.csv')

# 1. Limpieza de datos: eliminar filas con valores nulos
eventos = eventos.dropna()
aficionados = aficionados.dropna()
promociones = promociones.dropna()

# 2. Combinación de DataFrames usando 'id_evento' como clave, especificando 'validate' para muchos a muchos
eventos_aficionados = pd.merge(eventos, aficionados, on='id_evento', how='inner', validate='many_to_many')
datos_completos = pd.merge(eventos_aficionados, promociones, on='id_evento', how='inner', validate='many_to_many')

# 3. Responder las preguntas de negocio

# a. ¿Cuál es el evento con mayor asistencia total?
asistencia_max = eventos['asistentes_totales'].max()  # Encontrar el valor máximo de asistentes
evento_max_asistencia = eventos[eventos['asistentes_totales'] == asistencia_max]  # Filtrar el evento con mayor asistencia
print("Evento con mayor asistencia:", evento_max_asistencia['nombre_evento'].values[0])

# b. ¿Qué rango de edad asiste más frecuentemente a los eventos?
aficionados['rango_edad'] = pd.cut(aficionados['edad'], 
                                   bins=[0, 18, 30, 40, 50, 60, 100], 
                                   labels=['0-18', '19-30', '31-40', '41-50', '51-60', '60+'])
# Utilizamos value_counts() y obtenemos el índice del máximo manualmente
rango_edad_frecuente = aficionados['rango_edad'].value_counts().sort_values(ascending=False).index[0]
print("Rango de edad más frecuente:", rango_edad_frecuente)

# c. ¿Qué medio publicitario ha generado mayor impacto en términos de asistencia?
datos_completos['impacto_promocion'] = datos_completos['asistentes_totales'] / datos_completos['presupuesto']
impacto_por_medio = datos_completos.groupby('medio_publicitario')['impacto_promocion'].mean()

# Para encontrar el medio publicitario con mayor impacto
impacto_max = impacto_por_medio.max()
medio_publicitario_mayor_impacto = impacto_por_medio[impacto_por_medio == impacto_max].index[0]
print("Medio publicitario con mayor impacto:", medio_publicitario_mayor_impacto)

# d. ¿Cuál es la ubicación de residencia que más aficionados aporta a los eventos?
aficionados_por_ubicacion = aficionados.groupby('ubicación_residencia')['id_aficionado'].count()

# Encontramos el valor máximo manualmente
max_aficionados = aficionados_por_ubicacion.max()
ubicacion_mas_aficionados = aficionados_por_ubicacion[aficionados_por_ubicacion == max_aficionados].index[0]
print("Ubicación con más aficionados:", ubicacion_mas_aficionados)

# 4. Exportación del DataFrame final a CSV
datos_completos.to_csv('/workspace/reporte_eventos.csv', index=False)


Evento con mayor asistencia: Evento 29
Rango de edad más frecuente: 19-30
Medio publicitario con mayor impacto: TV
Ubicación con más aficionados: Ciudad A
